In [1]:
import findspark
findspark.init()

from pyspark.mllib.recommendation import *
import random
from operator import *
from collections import defaultdict

In [2]:
from pyspark import SparkContext

In [3]:
sc = SparkContext('local', 'a2')

In [4]:
artistData=sc.textFile('artist_data_small.txt')

In [5]:
artistAlias=sc.textFile('artist_alias_small.txt')

In [6]:
userArtist=sc.textFile('user_artist_data_small.txt')

In [7]:
artistData1=artistData.map(lambda x: x.split("\t"))

In [8]:
artistAlias1=artistAlias.map(lambda x: x.split("\t"))

In [10]:
userArtistData1=userArtist.map(lambda x: x.split(" "))

In [12]:
artistData2=artistData1.map(lambda x: (int(x[0]), x[1]))
artistAlias2=artistAlias1.map(lambda x: (int(x[0]), x[1]))
userArtistData2=userArtistData1.map(lambda x: (int(x[0]),int(x[1]), int(x[2])))

In [20]:
uad=userArtistData2.map(lambda x: (x[0], (x[1], x[2]))).groupByKey()
userArtistDict=uad.map(lambda x: (x[0], dict(x[1])))
dictcalc=userArtistDict.map(lambda x: (x[0], sum(x[1].values()), sum(x[1].values())/len(x[1])))
top3=dictcalc.sortBy(keyfunc=lambda x: x[1], ascending=False).take(3)
print("User "+ str(top3[0][0])+" has a total play count of "+str(top3[0][1])+" and a mean count of "+str(int(top3[0][2])))
print("User "+ str(top3[1][0])+" has a total play count of "+str(top3[1][1])+" and a mean count of "+str(int(top3[1][2])))
print("User "+ str(top3[2][0])+" has a total play count of "+str(top3[2][1])+" and a mean count of "+str(int(top3[2][2])))

User 1059637 has a total play count of 674412 and a mean count of 1878
User 2064012 has a total play count of 548427 and a mean count of 9455
User 2069337 has a total play count of 393515 and a mean count of 1519


In [23]:
trainData, validateData, testData = userArtistData2.randomSplit([4, 4, 2], 13)
trainData.cache()
testData.cache()
validateData.cache()

PythonRDD[32] at RDD at PythonRDD.scala:48

In [25]:
trainData.take(3)

[(1059637, 1000049, 1), (1059637, 1000056, 1), (1059637, 1000114, 2)]

In [26]:
validateData.take(3)

[(1059637, 1000010, 238), (1059637, 1000062, 11), (1059637, 1000123, 2)]

In [27]:
testData.take(3)

[(1059637, 1000094, 1), (1059637, 1000112, 423), (1059637, 1000113, 5)]

In [28]:
trainData.count()

19761

In [29]:
validateData.count()

19862

In [30]:
testData.count()

9858

In [59]:
def modelEval(model, dataset):
    artistlist=userArtistData2.map(lambda x: (x[1])).distinct()
    user1=userArtistData2.map(lambda x: (x[0])).distinct()
    userlist=user1.collect()
    user2=user1.cartesian(artistlist)

    training = trainData.map(lambda r : (r[0], r[1])).distinct()
    pred = user2.subtract(training)
    
    predict = model.predictAll(pred)
    tot=0.0
    
    valtest= dataset.map(lambda r: (r[0], r[1])).distinct()
    
    for user in userlist:
        train1 = predict.filter(lambda x : x[0] == user)
        valtest1 = valtest.filter(lambda x : x[0] == user)
        topx = sc.parallelize(train1.takeOrdered(valtest1.count(), lambda x: -x[2])).map(lambda x: (x[0], x[1]))

        mat = topx.intersection(valtest1)
        tot += float(mat.count())/float(valtest1.count())
    return (tot/user1.count())

In [ ]:
rankList = [2,10,20]
for r in rankList:
    model = ALS.trainImplicit(trainData, rank=r, seed=345)
    me=modelEval(model,validateData)
    print("The model score for rank " + str(r) + " is " + str(me))

In [ ]:
bestModel = ALS.trainImplicit(trainData, rank=10, seed=345)
modelEval(bestModel, testData)

In [55]:
recofive=bestModel.recommendProducts(1059637, 5)
count=0
for art in recofive:
    print("Artist "+ str(count) + ": " + artistData1.collectAsMap().get(str(art[1])))
    count=count+1

Artist 0: My Chemical Romance
Artist 1: Something Corporate
Artist 2: Evanescence
Artist 3: Alanis Morissette
Artist 4: Counting Crows
